# Protein generator

### Import libraries

In [231]:
import urllib
import pandas as pd
from IPython.display import clear_output

### Function of retrieve conditional tags and sequences for a pfam_id from for uniprot

In [423]:
def get_sequence_and_conditional_tags(pfam_id):
    with urllib.request.urlopen(("https://www.uniprot.org/uniprot/?query=database%3A%28type%3Apfam+"+str(pfam_id)+"%29&columns=id,organism,keywords,go(molecular%20function),go(cellular%20component),go(biological%20process),sequence&format=tab")) as f:
        query = f.read()
        query = str(query, 'utf-8')
    lines = query.split('\n')
    li_results = [x.split('\t') for x in lines]
    names = li_results[0]
    results = li_results[1:]
    return names, results

### Download sequences, keywords and gene ontologies from uniprot

In [427]:
pfam_id = 'pf00123'
names, results = get_sequence_and_conditional_tags(pfam_id)

In [435]:
df = pd.DataFrame(results, columns=names)
df = df.dropna()

In [436]:
df

,Entry,Organism,Keywords,Gene ontology (molecular function),Gene ontology (cellular component),Gene ontology (biological process),Sequence
0,P09684,Gadus morhua (Atlantic cod),Direct protein sequencing;Hormone;Secreted,hormone activity [GO:0005179],extracellular region [GO:0005576],mRNA stabilization [GO:0048255]; prolactin sec...,HSDAVFTDNYSRFRKQMAAKKYLNS
1,Q29W19,Bos taurus (Bovine),Amidation;Cleavage on pair of basic residues;H...,neuropeptide hormone activity [GO:0005184]; pe...,extracellular region [GO:0005576],adenylate cyclase-activating G protein-coupled...,MTMCSGARLALLVYGILMHSSVYGSPAASGLRFPGIRPENEVYDED...
2,P01280,Gallus gallus (Chicken),Amidation;Direct protein sequencing;Hormone;Re...,digestive hormone activity [GO:0046659]; hormo...,extracellular space [GO:0005615],cellular water homeostasis [GO:0009992]; diet ...,HSDGLFTSEYSKMRGNAQVQKFIQNLM
3,P07217,Ovis aries (Sheep),Amidation;Direct protein sequencing;Reference ...,hormone activity [GO:0005179],extracellular region [GO:0005576],,YADAIFTNSYRKILGQLSARKLLQDIMNRQQGERNQEQGAKVRL
4,Q60549,Mesocricetus auratus (Golden hamster),Amidation;Cleavage on pair of basic residues;R...,hormone activity [GO:0005179],extracellular region [GO:0005576],,MPLWVFFVILTLTNGSHCSPSPSLPFRIRRYADAIFTSSYRKVLGQ...
...,...,...,...,...,...,...,...
2272,A0A2R9AEX1,Pan paniscus (Pygmy chimpanzee) (Bonobo),Reference proteome;Signal,hormone activity [GO:0005179]; peptide hormone...,extracellular region [GO:0005576],adenylate cyclase-activating G protein-coupled...,MDTRNKAQLLVLLTLLSVLFSQTSAWPLYRAPSALRLGDRIPFEGA...
2273,A0A2U3V0I5,Tursiops truncatus (Atlantic bottle-nosed dolp...,Reference proteome;Signal,hormone activity [GO:0005179],extracellular region [GO:0005576],,MLLWVFFLVTLTLSSGSHGSPPSQPLRMPRYADAIFTNSYRKVLAQ...
2274,A0A673V4D0,Suricata suricatta (Meerkat),Reference proteome;Signal,pituitary adenylate cyclase activating polypep...,extracellular region [GO:0005576],G protein-coupled receptor signaling pathway [...,MTMCSGVRLALLVYGIIMHSSVYCSPAAAGLRFPGIRPDDEAYDED...
2275,A0A0D9RSY0,Chlorocebus sabaeus (Green monkey) (Cercopithe...,Reference proteome;Signal,hormone activity [GO:0005179]; identical prote...,cytoplasm [GO:0005737]; extracellular space [G...,adenylate cyclase-activating G protein-coupled...,MKSIYFVAVLFVMLVQGSWQRSLQDTEEKPRSVSASQTDPLGDPDQ...
